In [1]:
# Add backend to path to so we can acess all our code
import sys
sys.path.append('../')  # Adds backend/ to path

In [2]:
# Import the yfinance pipeline
from sec_data_pipeline.yfinance.yfinance_pipeline import YfinancePipeline

In [3]:
pipeline = YfinancePipeline()

In [4]:
tickers = pipeline._scrape_nasdaq_tickers()

In [ ]:
new_tickers = pipeline.validate_tickers(tickers)

Validating tickers:   0%|          | 0/5100 [00:00<?, ?it/s]/home/caden/Projects/AlgoFlow/backend/notebooks/../sec_data_pipeline/yfinance/yfinance_pipeline.py:340: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
Validating tickers:   0%|          | 1/5100 [00:00<41:56,  2.03it/s]/home/caden/Projects/AlgoFlow/backend/notebooks/../sec_data_pipeline/yfinance/yfinance_pipeline.py:340: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
Validating tickers:   0%|          | 2/5100 [00:00<28:58,  2.93it/s]/home/caden/Projects/AlgoFlow/backend/notebooks/../sec_data_pipeline/yfinance/yfinance_pipeline.py:340: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
Validating tickers:   0%|          | 3/5100 [00:00<24:59,  3.40it/s]/home/caden/Projects/AlgoFlow/backend/notebooks/../sec_data_pipeline/yfinance/yfinance_pipeline.py:340: FutureWarning: YF.do

In [6]:
len(new_tickers)

4

In [7]:
new_tickers

{'valid': [],
 'invalid': ['AACB',
  'AACBR',
  'AACBU',
  'AACG',
  'AACI',
  'AACIU',
  'AACIW',
  'AADR',
  'AAL',
  'AALG',
  'AAME',
  'AAOI',
  'AAON',
  'AAPB',
  'AAPD',
  'AAPG',
  'AAPL',
  'AAPU',
  'AARD',
  'AAUS',
  'AAVM',
  'AAXJ',
  'ABAT',
  'ABCL',
  'ABCS',
  'ABEO',
  'ABI',
  'ABIG',
  'ABL',
  'ABLLL',
  'ABLV',
  'ABLVW',
  'ABNB',
  'ABOS',
  'ABP',
  'ABPWW',
  'ABSI',
  'ABTC',
  'ABTS',
  'ABUS',
  'ABVC',
  'ABVE',
  'ABVEW',
  'ABVX',
  'ACAD',
  'ACB',
  'ACDC',
  'ACET',
  'ACFN',
  'ACGL',
  'ACGLN',
  'ACGLO',
  'ACHC',
  'ACHV',
  'ACIC',
  'ACIU',
  'ACIW',
  'ACLS',
  'ACLX',
  'ACMR',
  'ACNB',
  'ACNT',
  'ACOG',
  'ACON',
  'ACONW',
  'ACRS',
  'ACRV',
  'ACT',
  'ACTG',
  'ACTU',
  'ACWI',
  'ACWX',
  'ACXP',
  'ADAG',
  'ADAM',
  'ADAMG',
  'ADAMH',
  'ADAMI',
  'ADAML',
  'ADAMM',
  'ADAMN',
  'ADAMZ',
  'ADAP',
  'ADBE',
  'ADBG',
  'ADEA',
  'ADGM',
  'ADI',
  'ADIL',
  'ADMA',
  'ADN',
  'ADNWW',
  'ADP',
  'ADPT',
  'ADSE',
  'ADSEW',
  'A